In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [2]:
# Coletando os dados:
exp_2022 = pd.read_csv("..\Dados\COMEX\EXP_2022_MUN.csv", delimiter=";")
agropecuaria = pd.read_excel(r"..\Dados\COMEX\sh4_agropecuaria.xlsx")
tabela_final = pd.read_csv(r"..\Dados\Tabela_final\tabela_final.csv")
municipio_escolhidos = pd.read_csv(r"..\Dados\Tabela_final\municipios_potenciais.csv", index_col=0)


# Trantando os dados de exportação:
exp_2022 = exp_2022.merge(agropecuaria, how="inner", left_on="SH4", right_on="Código SH4")
exp_2022.drop(columns=["Código SH4"], inplace=True)

# Agrupamento que retorna a média do KG_Líquido exportado e do Valor de fobização:
exp_2022 = exp_2022.groupby(["CO_MUN", "Denominação nesse estudo"]).agg({"VL_FOB":"mean", "KG_LIQUIDO":"mean"}).reset_index()
exp_2022_sertao = exp_2022[exp_2022["CO_MUN"].isin(tabela_final["IBGE7"])]

# Index para ser os códigos dos municípios:
exp_2022_sertao.set_index("CO_MUN", inplace=True)
municipio_escolhidos.set_index("IBGE7", inplace=True)
tabela_final.set_index("IBGE7", inplace=True)


exp_2022_sertao = exp_2022_sertao.merge(tabela_final[["NOME"]], how="inner", left_index=True, right_index=True)
exp_2022_sertao.reset_index(inplace=True)

In [5]:
tabela_final

,UF,NOME,LATITUDE,LONGITUDE,DIST_EF_PUB,DIST_EM_PUB,IDEB_AI,IDEB_AF,DOCSUP_EF_PUB,DOCSUP_EM_PUB,...,T_ANALF15M,T_ANALF18A24,T_ANALF18M,T_ANALF25A29,T_ANALF25M,QUAL_MED_AGUA,DIST_CORPO_AGUA,TRANSPORT_COST,TOT_ESC_POR_POP,TOT_MAT_POR_POP
IBGE7,,,,,,,,,,,,,,,,,,,,,
2700300,AL,ARAPIRACA,-9.75487,-36.6615,28.190028,42.715813,4.682072,3.682635,54.40,56.96,...,22.45,6.27,24.10,11.76,28.61,62.439167,42.035186,24042.166072,0.000433,0.245423
2700706,AL,BATALHA,-9.67420,-37.1330,30.940427,36.601658,4.488950,3.591572,12.62,41.94,...,30.84,8.85,33.44,20.53,40.54,65.300843,24.642419,24257.110578,0.001200,0.384829
2700904,AL,BELO MONTE,-9.82272,-37.2770,34.584857,53.005007,4.091777,3.176668,27.12,31.84,...,38.11,12.42,41.81,30.29,49.99,65.300843,3.297594,24226.863743,0.002235,0.298063
2701209,AL,CACIMBINHAS,-9.40121,-36.9911,39.071078,42.736296,4.085816,3.310474,49.04,45.22,...,40.61,13.56,43.81,30.14,51.12,65.300843,58.541833,24358.206538,0.001102,0.419781
2701605,AL,CANAPI,-9.11932,-37.5967,31.912620,36.839799,4.673732,3.635387,20.84,32.34,...,40.35,15.27,44.16,28.31,51.97,66.893107,68.942666,24734.221802,0.002032,0.366838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2806008,SE,RIBEIROPOLIS,-10.53570,-37.4380,33.124036,39.388836,4.178352,3.548260,64.68,77.28,...,28.50,6.08,30.60,11.60,36.26,64.911294,1.388835,23891.778664,0.000692,0.211261
2807006,SE,SAO MIGUEL DO ALEIXO,-10.38470,-37.3836,31.585497,47.837042,3.932061,3.191697,62.08,66.38,...,33.54,7.79,37.03,18.32,44.72,64.911294,16.508752,23954.902605,0.002280,0.196098
2807105,SE,SIMAO DIAS,-10.73870,-37.8097,35.961743,43.788264,4.292358,3.462631,61.38,82.96,...,26.78,7.36,28.90,14.39,34.16,65.088090,2.685846,23918.838452,0.001108,0.254938


In [12]:
# Função para calcular a distância em KM, entre duas coordenadas geográficas:
def distancia_circular(df_escolhidos, df_municipios_sertao, threhold):
    dicionario_distancias = {}
    lista_distancias = []
    for municipio_escolhido, row in df_escolhidos[["LATITUDE", "LONGITUDE"]].iterrows():
        for municipio_qualquer, row2 in df_municipios_sertao[["LATITUDE", "LONGITUDE"]].iterrows():
            if municipio_escolhido != municipio_qualquer:
                distancia = geodesic(row.values, row2.values).kilometers
                if distancia <= threhold:
                    lista_distancias.append((municipio_qualquer, distancia))
    
        dicionario_distancias[df_escolhidos.loc[municipio_escolhido, "NOME"]] = lista_distancias
        lista_distancias = []

    return dicionario_distancias



# Função que determina, quais os produtos os municípios vizinhos estão exportando:
def produtos_exportados_vizinhos(dicionario, expor):
    dicionario_exportacao = {}
    lista_exportacao = []
    contagem = 0
    
    for key, value in dicionario.items():
        for municipio_vizinho in value:
            if  expor.isin([municipio_vizinho[0]]).any().any():
                contagem += 1
                nome = expor[expor["CO_MUN"] == municipio_vizinho[0]]["NOME"].values
                produto = expor[expor["CO_MUN"] == municipio_vizinho[0]]["Denominação nesse estudo"].values
                kg_liq = expor[expor["CO_MUN"] == municipio_vizinho[0]]["KG_LIQUIDO"].values
                valor_fb = expor[expor["CO_MUN"] == municipio_vizinho[0]]["VL_FOB"].values
                
                lista_exportacao.append((municipio_vizinho[0], nome, produto, kg_liq, valor_fb))
                display(expor[expor["CO_MUN"] == municipio_vizinho[0]])

        if contagem:
            print(f"{key}\n")
            dicionario_exportacao[key] = lista_exportacao
        contagem = 0
        lista_exportacao = []
        
    return dicionario_exportacao

### Hipótese 1: Nos municípios selecionados, há exportação de algum tipo de produto.

In [15]:
expo_municipios_escolhidos = municipio_escolhidos.merge(exp_2022_sertao, left_index=True, right_index=True, how="inner")
expo_municipios_escolhidos

,UF,NOME_x,LATITUDE,LONGITUDE,DIST_EF_PUB,DIST_EM_PUB,IDEB_AI,IDEB_AF,DOCSUP_EF_PUB,DOCSUP_EM_PUB,...,DIST_CORPO_AGUA,TRANSPORT_COST,TOT_ESC_POR_POP,TOT_MAT_POR_POP,CLUSTER,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO,NOME_y


Conclusão:

- Não há exportação de nenhum produto nos municípios escolhidos, bseado nos dados de exportação de 2022.

### Hipótese 2: Nos municípios vizinhos (50km), há exportação de algum tipo de produto.

In [14]:
# Rodando as funções:
distancias_municipios_escolhidos = distancia_circular(municipio_escolhidos, tabela_final, 50)
dicionario_export = produtos_exportados_vizinhos(distancias_municipios_escolhidos, exp_2022_sertao)

,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO,NOME
94,2922904,Sumos de frutas (incluídos os mostos de uvas) ...,248108.545455,82893.25,NOVA SOURE


,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO,NOME
64,2807402,Coprodutos do Algodão,22545.0,1526.0,TOBIAS BARRETO


ITAPICURU



,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO,NOME
55,2606002,Coprodutos do milho,29500.8,33765.2,GARANHUNS
56,2606002,Farelo/Farinha e Resíduos Sólidos,29500.8,33765.2,GARANHUNS


CAETES



...

BREJO DA MADRE DE DEUS



,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO,NOME
55,2606002,Coprodutos do milho,29500.8,33765.2,GARANHUNS
56,2606002,Farelo/Farinha e Resíduos Sólidos,29500.8,33765.2,GARANHUNS


BOM CONSELHO



,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO,NOME
55,2606002,Coprodutos do milho,29500.8,33765.2,GARANHUNS
56,2606002,Farelo/Farinha e Resíduos Sólidos,29500.8,33765.2,GARANHUNS


SAO JOAO

